## **A/B-Testing (Unabhängiger Zwei-Stichproben-t-Test)** 
### Geschäftsproblem
Digitale Werbeplattformen versuchen, die Konversionsraten zu optimieren, indem sie Werbetreibenden unterschiedliche Gebotsstrategien anbieten. Vor kurzem wurde als Alternative zum bestehenden "Maximum Bidding"-Modell eine neue Methode namens "Average Bidding" eingeführt.

Einer unserer Kunden, veridunya.com, hat beschlossen zu testen, ob dieses neue Gebotsmodell effizienter ist. Um zu verstehen, ob Average Bidding im Vergleich zu Maximum Bidding eine höhere Konversionsrate erzielt, wurde die Durchführung eines A/B-Tests angefordert.

Dieser A/B-Test läuft seit einem Monat, und veridunya.com erwartet von Ihnen eine Analyse der Testergebnisse. Die wichtigste Erfolgskennzahl für das Unternehmen ist die Kaufmetrik (Purchase). Daher sollten sich die statistischen Analysen auf diese Metrik konzentrieren.
### Datensatz
Dieser Datensatz enthält Werbeeinblendungen und Nutzerinteraktionen einer E-Commerce-Website.
Er umfasst Informationen über die Anzahl der Klicks auf angezeigte Werbeanzeigen sowie die darauserzielten Einnahmen. 
In der Analyse gibt es zwei unterschiedliche Gruppen:  
**Kontrollgruppe:** Es wurde die Methode des Maximalgebots (Maximum Bidding) angewendet.  
**Testgruppe:** Es wurde die Methode des Durchschnittsgebots (Average Bidding) angewendet.  
  
**Impression:** Anzahl der Werbeeinblendungen   
**Click:** Anzahl der Klicks auf die angezeigte Werbung  
**Purchase:** Anzahl der Käufe nach dem Klick auf die Werbung  
**Earning:** Einnahmen nach dem Kauf

**AUFGABE 1:** Datenvorbereitung und Datenanalyse

In [2]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#%pip install statsmodels
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Schritt 1: Die Daten der Kontroll- und Testgruppen werden eingelesen und separaten Variablen zugeordnet.
df_control= pd.read_excel("C:/Users/nesri/repos/AB_testing/AB_Testing_data.xlsx",sheet_name="Control Group")
df_test = pd.read_excel("C:/Users/nesri/repos/AB_testing/AB_Testing_data.xlsx", sheet_name="Test Group")

In [3]:
df_control.head()

,Impression,Click,Purchase,Earning
0,82529.45927,6090.07732,665.21125,2311.27714
1,98050.45193,3382.86179,315.08489,1742.80686
2,82696.02355,4167.96575,458.08374,1797.82745
3,109914.40040,4910.88224,487.09077,1696.22918
4,108457.76263,5987.65581,441.03405,1543.72018


In [4]:
df_test.head()

,Impression,Click,Purchase,Earning
0,120103.50380,3216.54796,702.16035,1939.61124
1,134775.94336,3635.08242,834.05429,2929.40582
2,107806.62079,3057.14356,422.93426,2526.24488
3,116445.27553,4650.47391,429.03353,2281.42857
4,145082.51684,5201.38772,749.86044,2781.69752


In [5]:
df_control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


In [ ]:
#Schritt 2: Grundlegende Statistik der Kontroll- und Testgruppendaten

In [7]:
df_control.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,101711.44907,20302.15786,45475.94296,85726.69035,99790.70108,115212.81654,147539.33633
Click,40.00000,5100.65737,1329.98550,2189.75316,4124.30413,5001.22060,5923.80360,7959.12507
Purchase,40.00000,550.89406,134.10820,267.02894,470.09553,531.20631,637.95709,801.79502
Earning,40.00000,1908.56830,302.91778,1253.98952,1685.84720,1975.16052,2119.80278,2497.29522


In [8]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,120512.41176,18807.44871,79033.83492,112691.97077,119291.30077,132050.57893,158605.92048
Click,40.00000,3967.54976,923.09507,1836.62986,3376.81902,3931.35980,4660.49791,6019.69508
Purchase,40.00000,582.10610,161.15251,311.62952,444.62683,551.35573,699.86236,889.91046
Earning,40.00000,2514.89073,282.73085,1939.61124,2280.53743,2544.66611,2761.54540,3171.48971


In [10]:
#Schritt 3: Die Zusammenführung der Daten der Kontroll- und Testgruppe erfolgt unter Verwendung der concat-Methode.
df_control["group"] = "control"
df_test["group"] = "test"

df = pd.concat([df_control, df_test], axis=0, ignore_index=False)
df.head()

,Impression,Click,Purchase,Earning,group
0,82529.45927,6090.07732,665.21125,2311.27714,control
1,98050.45193,3382.86179,315.08489,1742.80686,control
2,82696.02355,4167.96575,458.08374,1797.82745,control
3,109914.40040,4910.88224,487.09077,1696.22918,control
4,108457.76263,5987.65581,441.03405,1543.72018,control


In [11]:
df.sort_values(by="group")

,Impression,Click,Purchase,Earning,group
0,82529.45927,6090.07732,665.21125,2311.27714,control
22,83676.60243,7153.97419,487.82877,1989.64192,control
23,80254.33164,3075.31120,530.67988,2101.31948,control
24,123961.86872,4898.78841,585.63917,2042.18293,control
25,94472.19659,5937.47947,686.41229,1616.58901,control
...,...,...,...,...,...
14,119877.96005,3622.93635,689.15574,2811.50273,test
15,137222.38146,4042.27828,677.27270,2260.03463,test
16,134387.16892,4986.10073,417.99034,2087.98827,test
18,115934.85439,5059.85845,653.49152,2682.82044,test


**AUFGABE 2:** Definition der Hypothesen für den A/B-Test

In [15]:
# Schritt 1: Die Definition der Hypothesen

# H0 : M1 = M2 
# (Es gibt keinen Unterschied zwischen den durchschnittlichen Kaufzahlen der Kontrollgruppe und der Testgruppe.)

# H1 : M1 != M2 
# (Es gibt einen Unterschied zwischen den durchschnittlichen Kaufzahlen der Kontrollgruppe und der Testgruppe.)

# H0: Es besteht kein statistisch signifikanter Unterschied zwischen der durchschnittlichen Anzahl an Käufen der Kontrollgruppe 
# (mit der "Maximum Bidding"-Kampagne) und der Testgruppe (mit der "Average Bidding"-Kampagne).

# H1: Es besteht ein statistisch signifikanter Unterschied zwischen der durchschnittlichen Anzahl an Käufen der Kontrollgruppe 
# (mit der "Maximum Bidding"-Kampagne) und der Testgruppe (mit der "Average Bidding"-Kampagne).

# Schritt 2: Die Berechnung der durchschnittlichen Kaufzahlen (Purchase) der Kontroll- und Testgruppe

df.groupby("group").agg({"Purchase": "mean"})

,Purchase
group,
control,550.89406
test,582.10610


**AUFGABE 3:** Durchführung des Hypothesentests

In [16]:
#Schritt 1: Kontrolle der Annahmen vor dem Hypothesentest

# Test der Normalverteilungsannahme für Kontrollgruppe und Testgruppe basierend auf der Variable "Purchase"

# Normalverteilungsannahme:
# H0: Die Normalverteilungsannahme ist erfüllt.
# H1: Die Normalverteilungsannahme ist nicht erfüllt.
# p < 0.05 → H0 wird verworfen
# p > 0.05 → H0 kann nicht verworfen werden

test_stat, pvalue = shapiro(df.loc[df["group"] == "control", "Purchase"])
print('Teststatistik = %.4f, p-Wert = %.4f' % (test_stat, pvalue))
# p-Wert=0.5891
# H0 kann nicht verworfen werden. Die Werte der Kontrollgruppe erfüllen die Annahme der Normalverteilung.

Teststatistik = 0.9773, p-Wert = 0.5891


In [17]:
test_stat, pvalue = shapiro(df.loc[df["group"] == "test", "Purchase"])
print('Teststatistik = %.4f, p-Wert = %.4f' % (test_stat, pvalue))
# p-Wert=0.1541
# H0 kann nicht verworfen werden. Die Werte der Testgruppe erfüllen die Annahme der Normalverteilung.

Teststatistik = 0.9589, p-Wert = 0.1541


In [21]:
# Varianzhomogenität:
# H0: Die Varianzen sind homogen.
# H1: Die Varianzen sind nicht homogen.
# p < 0.05 → H0 wird verworfen
# p > 0.05 → H0 kann nicht verworfen werden

test_stat, pvalue = levene(df.loc[df["group"] == "control", "Purchase"],
                           df.loc[df["group"] == "test", "Purchase"])


print('Teststatistik= %.4f, p-Wert = %.4f' % (test_stat, pvalue))
# p-Wert=0.1083
# H0 kann nicht verworfen werden. 
# Die Werte der Kontrollgruppe und Testgruppe erfüllen die Annahme der Varianzhomogenität.
# Die Varianzen sind homogen.

Teststatistik= 2.6393, p-Wert = 0.1083


In [23]:
#Schritt 2: Auswahl des geeigneten statistischen Tests
# Da sowohl die Normalverteilungsannahme als auch die Varianzhomogenität erfüllt sind,
# wird ein unabhängiger Zwei-Stichproben-t-Test (parametrischer Test) durchgeführt.

# H0: M1 = M2
# (Es gibt keinen statistisch signifikanten Unterschied zwischen den Kaufdurchschnitten
# der Kontroll- und der Testgruppe.)
#
# H1: M1 ≠ M2
# (Es gibt einen statistisch signifikanten Unterschied zwischen den Kaufdurchschnitten
# der Kontroll- und der Testgruppe.)
#
# p < 0.05 → H0 wird verworfen
# p > 0.05 → H0 kann nicht verworfen werden

test_stat, pvalue = ttest_ind(df.loc[df["group"] == "control", "Purchase"],
                              df.loc[df["group"] == "test", "Purchase"],
                              equal_var=True)

print('Teststatistik = %.4f, p-Wert = %.4f' % (test_stat, pvalue))


Teststatistik = -0.9416, p-Wert = 0.3493


In [ ]:
#Schritt 3: Interpretation der Testergebnisse
## p-Wert = 0.3493
# H0 kann nicht verworfen werden. Es besteht kein statistisch signifikanter Unterschied zwischen 
# den durchschnittlichen Kaufbeträgen der Kontroll- und der Testgruppe.

**AUFGABE 4:** Bewertung der Ergebnisse

In [ ]:
# Schritt 1: Verwendeter Test und Begründung

Es wurde der **unabhängige t-Test** verwendet, da festgestellt werden sollte, ob zwischen den Mittelwerten **zwei unabhängiger Gruppen** ein signifikanter Unterschied in bestimmten Merkmalen besteht.  
Nach Überprüfung der Annahmen wurde festgestellt, dass **beide Gruppen normalverteilt** sind und **Varianzhomogenität** vorliegt.

In [ ]:
# Schritt 2: Empfehlung für den Kunden basierend auf den Testergebnissen

Im Hinblick auf **Käufe (Purchase)** konnte kein signifikanter Unterschied festgestellt werden, das bedeutet, **H0 kann nicht verworfen werden**.  
Es besteht kein statistisch signifikanter Unterschied zwischen den durchschnittlichen Kaufzahlen der **Kontroll- und Testgruppe**.  

Daher kann der Kunde **beide Angebotsstrategien (Maximum Bidding und Average Bidding)** weiterhin parallel einsetzen.  

Allerdings sollten auch Unterschiede in **anderen Kennzahlen** berücksichtigt werden, um die profitabelste Strategie zu identifizieren:  
- **Klicks (Click-Through)**  
- **Interaktionen (Engagement)**  
- **Einnahmen (Revenue)**  
- **Konversionsraten (Conversion Rate)**  

Zusätzlich könnte es sinnvoll sein, die **Dauer des Tests zu verlängern**, um robustere Ergebnisse zu erhalten.